# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [39]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [40]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [41]:
# We know that j3h145-jcop3 card supports T1 protocol, so we use that ignoring ATR
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [42]:
atr=bytes(connection.getATR())
atr.hex()

'3bdc18ff8191fe1fc38073c821136605036351000250'

## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [43]:
def select(appletID):
    """
    Selects an applet by AppletID (should be hex-encoded string).
    Throws an error with the error code if it failed selecting the applet.
    Useful link for error codes: https://www.eftlab.com/knowledge-base/complete-list-of-apdu-responses/
    """
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print("Sent command:", bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [44]:
# MemoryCardApplet has AID of `B00B5111CB01`
select("B00B5111CB01")

Sent command: 00A4040006B00B5111CB0100


b''

In [45]:
def request(APDU):
    """
    Sends APDU command to the card and returns data.
    APDU should be hex-encoded.
    Throws an error with the error code if it failed selecting the applet.
    Useful link for error codes: https://www.eftlab.com/knowledge-base/complete-list-of-apdu-responses/
    """
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

# 2. Establish secure communication
This part is required to establish a shared secret between the card and the host.<br>
Because we want encrypted and authenticated communication.<br>

If you already agreed with the card on a shared secret you can skip this step and go to step 3.<br>
If the card was communicating with someone else it probably forgot your shared secret, so 
you may need to run through this procedure again.

Same happens if the applet is reinstalled - all the data of the applet is wiped, so it's 
like communicating with a fresh new card.

For key agreement we use ECDH - get card's pubkey, multiply by our secret, use x-coord of the resulting point as shared secret. Maybe sha256 of it?

## 2.1. Get public key from the card

In [46]:
# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1

In [47]:
# get pubkey - private key is generated when applet is installed
sec = request("B0A1000000")
card_pubkey = PublicKey.parse(sec)
card_pubkey

PublicKey(04c1a0f791d338b9d147a6b9369ba93075cb9fe574256c92e912643cf34fe3a8cb5f4f6ad6b3f639e958563403219752a747bcc4368599fc87cc05fe8ce940e0dd)

## 2.2. Select a random secret on the host

In [48]:
import os
import hashlib
# secret = os.urandom(32)
secret = b'FC\xe2\t\x1bd\xb9~H\xe7\x1dF\xa5\x88W+g\x1b-\xc5A\xec\x19\x00\xe8&\x8d\xa3\x13\xec?\x88'
# ugly copy...
pub = PublicKey.parse(card_pubkey.sec())
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
shared_hash = hashlib.sha256(shared_secret).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: d89dc62098d5e0898eb1746f06e2ba835e2f34784be3240eb867f4f6825a2172
Sha256 of the secret: 9f984986bbb54c70cd5608273f8114f144d406483cac5587ece37aad9a46ab82


In [49]:
host_prv = PrivateKey(secret)
host_pub = host_prv.get_public_key()
host_pub.compressed = False
data = bytes([65])+host_pub.sec()
data.hex()

'4104641803d1e6e42c0103c39f6917fd66a3e443824749e78f7fafb931ad6985a23398aabeb9e20f3bc32ab369994713ac2265328c0a01c0648409a935b133b71b8e'

In [50]:
res = request("B0A30000"+data.hex())
if res == shared_hash:
    print("Yey! We have a shared secret now!")
else:
    print("Meh... Something didn't work? Card returned %s, we have %s" % (res.hex(), shared_hash.hex()))

Yey! We have a shared secret now!


In [ ]:
# get random data
rand = request("B0A2000000")
rand.hex()

In [ ]:
# static data for any other APDU
data = request("B000000000")
data

In [38]:
connection.disconnect()